<a href="https://colab.research.google.com/github/Hieuxuan1112/bilingual-document-retrieval/blob/main/02_internal_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Environment Setup & Dependencies

In [15]:
!pip install -q pymupdf sentence-transformers google-generativeai pdfplumber python-docx
!pip install -q scikit-learn beautifulsoup4 requests nltk

## 2. Configuration & Model Initialization

In [16]:
import nltk
import requests
import fitz  # pymupdf
import io
import os
from PIL import Image
import docx
import google.generativeai as genai
from sentence_transformers import SentenceTransformer, util
from google.colab import userdata # <--- Quan trọng: Thư viện để đọc Secrets

# Tải gói dữ liệu cần thiết cho nltk
nltk.download("punkt", quiet=True)

# --- CẤU HÌNH (LẤY TỪ SECRETS) ---
try:
    API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=API_KEY)
    print("✅ Đã lấy API Key thành công từ Secrets.")
except ImportError:
    print("⚠️ Lỗi: Không tìm thấy thư viện google.colab (Chạy trên local?).")
except Exception as e:
    print("⚠️ CHƯA CẤU HÌNH SECRETS! Hãy thêm 'GEMINI_API_KEY' vào biểu tượng Chìa khóa bên trái.")

model_gemini = genai.GenerativeModel('gemini-flash-latest')

# Load model so sánh câu
labse_model = SentenceTransformer("sentence-transformers/LaBSE")

print("Đã cấu hình xong Model: gemini-flash-latest")

✅ Đã lấy API Key thành công từ Secrets.
Đã cấu hình xong Model: gemini-flash-latest


## 3. Gemini Helper Functions (Content Analysis)

In [17]:
import re

def get_info_from_gemini(input_data, data_type):
    """
    Hàm cải tiến (V2): Lấy cả TỪ KHÓA (để search) và NỘI DUNG (để so khớp LaBSE).
    Trả về: (keywords_list, content_text_for_labse)
    """
    if not input_data: return [], ""

    # Prompt mới: Yêu cầu Gemini vừa OCR/Dịch vừa sinh từ khóa và định dạng rõ ràng
    base_prompt = """
    Bạn là chuyên gia Hán Nôm và Sử học. Hãy phân tích hình ảnh hoặc văn bản đầu vào.

    Yêu cầu 1 (Nội dung để so khớp):
    - Nếu là Ảnh chữ Hán/Nôm: Hãy OCR nhận dạng các chữ trong ảnh rồi dịch nghĩa vắn tắt sang Tiếng Việt.
    - Nếu là Ảnh Tiếng Việt/Văn bản: Hãy tóm tắt nội dung chính.
    -> Viết kết quả thành một đoạn văn xuôi tiếng Việt (khoảng 100-200 chữ).

    Yêu cầu 2 (Từ khóa tìm kiếm):
    - Đề xuất 5 từ khóa tìm kiếm (search queries) hiệu quả nhất để tìm tài liệu này trên Google.

    BẮT BUỘC TRẢ VỀ THEO ĐỊNH DẠNG SAU:
    ---CONTENT_START---
    (Viết đoạn văn nội dung ở đây)
    ---CONTENT_END---
    (Viết các từ khóa ở dưới đây, mỗi từ một dòng)
    """

    try:
        if data_type == 'image':
            # Gửi ảnh + Prompt
            response = model_gemini.generate_content([base_prompt, input_data])
        else:
            # Gửi Text + Prompt
            text_prompt = base_prompt + f"\n\nNội dung đầu vào:\n{input_data[:2000]}"
            response = model_gemini.generate_content(text_prompt)

        # Xử lý text trả về để tách Nội dung và Từ khóa
        full_response = response.text.strip()

        # 1. Tách nội dung giữa 2 thẻ marker
        extracted_content = ""
        keywords = []

        content_match = re.search(r'---CONTENT_START---(.*?)---CONTENT_END---', full_response, re.DOTALL)
        if content_match:
            extracted_content = content_match.group(1).strip()

            # 2. Lấy phần còn lại làm keywords
            keywords_raw = full_response.split('---CONTENT_END---')[-1].strip()
            keywords = [k.strip().strip('-').strip() for k in keywords_raw.split('\n') if k.strip()]
        else:
            # Dự phòng nếu Gemini quên format: Lấy dòng đầu làm content, các dòng sau làm keyword
            lines = full_response.split('\n')
            extracted_content = lines[0]
            keywords = lines[1:]

        return keywords, extracted_content

    except Exception as e:
        print(f"⚠️ Lỗi Gemini ({data_type}): {e}")
        return [], ""

## 4. Dataset Preparation

In [18]:
# Thêm tham số -o (overwrite) để tự động ghi đè file cũ
!unzip -qo Han_chapters.zip -d dataset_han
# Lưu ý: Nhìn ảnh của bạn thì tên file là Viet_chapers.zip (thiếu chữ t),
# nếu lệnh dưới lỗi thì bạn sửa thành Viet_chapers.zip nhé
!unzip -qo Viet_chapters.zip -d dataset_viet

print("Đã giải nén xong!")

Đã giải nén xong!


## 5. Online Search Utilities (Mode 2 - WIP)

In [19]:
def search_vietnamese_docs(sentences, max_results=5):
    # Lấy key SerpApi từ két sắt
    from google.colab import userdata
    try:
        serp_key = userdata.get('SERPAPI_KEY')
    except Exception:
        serp_key = ""
        print("⚠️ CẢNH BÁO: Chưa cấu hình 'SERPAPI_KEY' trong Secrets. Chức năng tìm kiếm online sẽ lỗi!")

    results = []
    for s in sentences:
        params = {
            "engine": "google",
            "q": s[:80] + " filetype:pdf",
            "hl": "vi",
            "gl": "vn",
            "api_key": serp_key  # <--- DÙNG BIẾN serp_key (AN TOÀN)
        }

        try:
            r = requests.get("https://serpapi.com/search", params=params)
            data = r.json()

            if "organic_results" in data:
                for res in data["organic_results"]:
                    if "link" in res:
                        results.append(res["link"])
            elif "error" in data:
                print(f"⚠️ Lỗi từ SerpApi: {data['error']}")

        except Exception as e:
            print(f"⚠️ Lỗi kết nối SerpApi: {e}")

    return list(set(results))[:max_results]

In [20]:
def download_files(urls):
    paths = []
    for i, url in enumerate(urls):
        try:
            r = requests.get(url, timeout=15)
            if r.status_code == 200 and len(r.content) > 50000:
                path = f"vi_doc_{i}.pdf"
                with open(path, "wb") as f:
                    f.write(r.content)
                paths.append(path)
        except:
            pass
    return paths


## 6. Core Logic (Hybrid Matching V5) & Main Interface

In [21]:
import os
import glob
import time
import requests
import fitz # PyMuPDF
from google.colab import files
from sentence_transformers import util
from PIL import Image
import re

# Thiết lập đường dẫn dữ liệu
HAN_FOLDER = "dataset_han"
VIET_FOLDER = "dataset_viet"

# ==============================================================================
# HÀM CHUYỂN ĐỔI TÊN FILE HÁN -> VIỆT (CHỈ ĐỂ HIỂN THỊ)
# ==============================================================================
def dich_ten_file_han(ten_file):
    tu_vung = {
        "外紀": "Ngoại Kỷ", "本紀": "Bản Kỷ", "續編": "Tục Biên",
        "卷": " Quyển ", "之": "", ".txt": ""
    }
    for han, viet in tu_vung.items():
        ten_file = ten_file.replace(han, viet)

    so_dem = {
        "二十": "20", "十九": "19", "十八": "18", "十七": "17", "十六": "16",
        "十五": "15", "十四": "14", "十三": "13", "十二": "12", "十一": "11",
        "十": "10", "九": "9", "八": "8", "七": "7", "六": "6",
        "五": "5", "四": "4", "三": "3", "二": "2", "一": "1"
    }
    for han, viet in so_dem.items():
        ten_file = ten_file.replace(han, viet)

    return ten_file.strip()

# ==============================================================================
# 1. HÀM GỌI GEMINI:
#    - Trích xuất từ khóa chính
#    - Dịch mẫu đoạn đầu để làm truy vấn ngữ nghĩa
# ==============================================================================
def analyze_content_with_gemini(han_text):
    """
    Sử dụng Gemini để:
    (1) Lấy danh sách thực thể / danh từ riêng quan trọng
    (2) Dịch thử một đoạn ngắn ở đầu văn bản sang tiếng Việt
    """
    chunk = han_text[:2500]

    prompt = f"""
    Bạn là trợ lý AI hỗ trợ tra cứu sử liệu. Hãy xử lý văn bản Hán Nôm sau:

    Nhiệm vụ 1: Liệt kê 10-15 DANH TỪ RIÊNG quan trọng nhất (Nhân vật, Địa danh, Niên hiệu) bằng tiếng Việt.
    Nhiệm vụ 2: Dịch một đoạn văn khoảng 200 chữ ở phần đầu văn bản sang tiếng Việt để làm mẫu so sánh.

    Văn bản Hán:
    {chunk}

    Trả về đúng định dạng JSON, không thêm chú thích:
    {{
        "keywords": ["Từ 1", "Từ 2", ...],
        "translation": "Nội dung dịch đoạn văn mẫu..."
    }}
    """
    try:
        response = model_gemini.generate_content(prompt)
        text_resp = response.text.replace("```json", "").replace("```", "").strip()
        import json
        data = json.loads(text_resp)
        return data.get("keywords", []), data.get("translation", "")
    except Exception as e:
        print(f"[Lỗi] Không thể xử lý phản hồi từ Gemini: {e}")
        return [], ""

# ==============================================================================
# 2. TÌM KIẾM HAI GIAI ĐOẠN:
#    - Giai đoạn 1: Lọc nhanh bằng từ khóa
#    - Giai đoạn 2: Xếp hạng lại bằng độ tương đồng ngữ nghĩa (LaBSE)
# ==============================================================================
def find_best_match_v5(source_file_path, target_folder):
    # Bước 1: Đọc nội dung văn bản Hán
    with open(source_file_path, 'r', encoding='utf-8') as f:
        han_content = f.read()

    print("Đang phân tích nội dung văn bản nguồn bằng Gemini...")
    keywords, translated_chunk = analyze_content_with_gemini(han_content)

    if not keywords:
        print("Không lấy được từ khóa từ Gemini.")
        return []

    print(f"Số từ khóa trích xuất được: {len(keywords)}")
    print(f"Đoạn dịch mẫu dùng để so khớp ngữ nghĩa (rút gọn): {translated_chunk[:80]}...")

    # Bước 2: Lọc sơ bộ các file tiếng Việt bằng từ khóa
    print("\nGiai đoạn 1: Lọc sơ bộ theo từ khóa")
    files = glob.glob(os.path.join(target_folder, "**/*.txt"), recursive=True)
    candidates = []

    for f_path in files:
        try:
            with open(f_path, 'r', encoding='utf-8') as f:
                tgt_content = f.read().lower()

            match_count = 0
            for kw in keywords:
                if kw.lower() in tgt_content:
                    match_count += 1

            if match_count > 0:
                score_kw = match_count / len(keywords)
                candidates.append((score_kw, f_path))
        except:
            continue

    candidates.sort(key=lambda x: x[0], reverse=True)
    top_candidates = candidates[:5]

    if not top_candidates:
        print("Không có file nào vượt qua vòng lọc từ khóa.")
        return []

    print(f"Chọn {len(top_candidates)} ứng viên vào vòng đánh giá ngữ nghĩa:")
    for s, f in top_candidates:
        print(f" - {os.path.basename(f)} | Mức khớp từ khóa: {s*100:.0f}%")

    # Bước 3: So khớp ngữ nghĩa bằng LaBSE
    print("\nGiai đoạn 2: So khớp ngữ nghĩa bằng embedding (LaBSE)")

    query_emb = labse_model.encode(translated_chunk, convert_to_tensor=True)
    final_results = []

    for kw_score, f_path in top_candidates:
        try:
            with open(f_path, 'r', encoding='utf-8') as f:
                tgt_content = f.read()

            if len(tgt_content) > 2000:
                target_segment = tgt_content[300:1800]
            else:
                target_segment = tgt_content

            tgt_emb = labse_model.encode(target_segment, convert_to_tensor=True)
            sem_score = util.pytorch_cos_sim(query_emb, tgt_emb).item()

            final_score = (kw_score * 0.3) + (sem_score * 0.7)
            final_results.append((final_score, f_path, sem_score, kw_score))
        except:
            continue

    final_results.sort(key=lambda x: x[0], reverse=True)
    return final_results

# ==============================================================================
# GIAO DIỆN CHƯƠNG TRÌNH (CLI)
# ==============================================================================
print("==============================================")
print("CHƯƠNG TRÌNH TÌM KIẾM TÀI LIỆU SONG NGỮ (V5)")
print("==============================================")
print("[1] Tìm kiếm trong bộ dữ liệu nội bộ")
print("[2] Tìm kiếm online (chức năng đang hoàn thiện)")
print("==============================================")

mode_choice = input("Chọn chế độ (1 hoặc 2): ").strip()

if mode_choice == '1':
    print("\nDanh sách các file văn bản Hán hiện có:")
    all_han_files = sorted(glob.glob(os.path.join(HAN_FOLDER, "**/*.txt"), recursive=True))

    if all_han_files:
        for i, f in enumerate(all_han_files):
            ten_goc = os.path.basename(f)
            print(f"[{i}] {ten_goc}  -->  {dich_ten_file_han(ten_goc)}")

        try:
            file_idx = int(input(f"\nChọn file cần đối chiếu (0 - {len(all_han_files)-1}): "))
            target_file = all_han_files[file_idx]
            print(f"\nFile đã chọn: {os.path.basename(target_file)}")

            results = find_best_match_v5(target_file, VIET_FOLDER)

            print("\nKết quả xếp hạng cuối cùng:")
            for i, (f_score, fname, sem_s, kw_s) in enumerate(results):
                print(f"Hạng {i+1}: {os.path.basename(fname)}")
                print(f"  - Điểm tổng hợp: {f_score*100:.2f}%")
                print(f"  - Ngữ nghĩa: {sem_s*100:.1f}% | Từ khóa: {kw_s*100:.1f}%")
        except Exception as e:
            print(f"Lỗi khi xử lý lựa chọn file: {e}")
    else:
        print("Không tìm thấy file Hán trong thư mục dữ liệu.")

elif mode_choice == '2':
    print("Chức năng tìm kiếm online hiện đang trong quá trình phát triển.")


CHƯƠNG TRÌNH TÌM KIẾM TÀI LIỆU SONG NGỮ (V5)
[1] Tìm kiếm trong bộ dữ liệu nội bộ
[2] Tìm kiếm online (chức năng đang hoàn thiện)
Chọn chế độ (1 hoặc 2): 1

Danh sách các file văn bản Hán hiện có:
[0] 外紀卷之一.txt  -->  Ngoại Kỷ Quyển 1
[1] 外紀卷之三.txt  -->  Ngoại Kỷ Quyển 3
[2] 外紀卷之二.txt  -->  Ngoại Kỷ Quyển 2
[3] 外紀卷之五.txt  -->  Ngoại Kỷ Quyển 5
[4] 外紀卷之四.txt  -->  Ngoại Kỷ Quyển 4
[5] 本紀卷之一.txt  -->  Bản Kỷ Quyển 1
[6] 本紀卷之七.txt  -->  Bản Kỷ Quyển 7
[7] 本紀卷之三.txt  -->  Bản Kỷ Quyển 3
[8] 本紀卷之九.txt  -->  Bản Kỷ Quyển 9
[9] 本紀卷之二.txt  -->  Bản Kỷ Quyển 2
[10] 本紀卷之五.txt  -->  Bản Kỷ Quyển 5
[11] 本紀卷之八.txt  -->  Bản Kỷ Quyển 8
[12] 本紀卷之六.txt  -->  Bản Kỷ Quyển 6
[13] 本紀卷之十.txt  -->  Bản Kỷ Quyển 10
[14] 本紀卷之十一.txt  -->  Bản Kỷ Quyển 11
[15] 本紀卷之十七.txt  -->  Bản Kỷ Quyển 17
[16] 本紀卷之十三.txt  -->  Bản Kỷ Quyển 13
[17] 本紀卷之十九.txt  -->  Bản Kỷ Quyển 19
[18] 本紀卷之十二.txt  -->  Bản Kỷ Quyển 12
[19] 本紀卷之十五.txt  -->  Bản Kỷ Quyển 15
[20] 本紀卷之十八.txt  -->  Bản Kỷ Quyển 18
[21] 本紀卷之十六.txt  -->  Bản Kỷ Quy